In [1]:
import os 
import numpy as np
import pandas as pd

In [2]:
DATA_FOLDER_PATH = os.path.join(os.pardir, 'data')
RAW_DATA_FOLDER_PATH = os.path.join(DATA_FOLDER_PATH, "raw")
PROCESSED_DATA_FOLDER_PATH = os.path.join(DATA_FOLDER_PATH, "processed")

In [ ]:
interactions.describe()

In [ ]:
df = pd.DataFrame(index=interactions['recipe_id'].unique(), columns=interactions['user_id'].unique())

In [ ]:
interactions['recipe_id'] = interactions['recipe_id'].astype('str')
interactions['user_id'] = interactions['user_id'].astype('str')

In [ ]:
interactions['recipe_id'].describe()

In [ ]:
interactions['user_id'].describe()

In [ ]:
interactions.head()

In [ ]:
interactions[interactions['user_id'] == '942504933']

In [39]:
# Item based approach
interactions.pivot(columns='recipe_id', index='user_id', values='rating')

IndexError: index 942541416 is out of bounds for axis 0 with size 942387538

In [4]:
def string_list_to_list(x):
    return x.strip('[]').replace('"', '').replace("'", '').replace(' ', '').split(',')

In [5]:
def create_features(x):
    """position 1 is name, 2 is ingredients and 3 is tags"""
    return [x[1]]+x[2]+x[3]

In [6]:
recipes = pd.read_csv(os.path.join(RAW_DATA_FOLDER_PATH, "RAW_recipes.csv"))
recipes['tags']=recipes['tags'].apply(lambda x: string_list_to_list(x))
recipes['ingredients']=recipes['ingredients'].apply(lambda x: string_list_to_list(x))
# recipes = recipes[['id','name','ingredients', 'tags']]
# cols = ['name','ingredients', 'tags']
# recipes['features'] = recipes.apply(lambda x: create_features(x), axis=1)
recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"[wintersquash, mexicanseasoning, mixedspice, h...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"[30-minutes-or-less, time-to-make, course, mai...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"[preparedpizzacrust, sausagepatty, eggs, milk,...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"[time-to-make, course, preparation, main-dish,...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"[groundbeef, yellowonions, dicedtomatoes, toma...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"[60-minutes-or-less, time-to-make, course, mai...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","[spreadablecheesewithgarlicandherbs, newpotato...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"[weeknight, time-to-make, course, main-ingredi...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"[tomatojuice, applecidervinegar, sugar, salt, ...",8


In [7]:
t = recipes.head()

# t['features'][0]

In [8]:
from lightfm.data import Dataset

D:\Miniconda\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [9]:
len(interactions['user_id'].unique())

226570

In [10]:
len(interactions['recipe_id'].unique())

231637

In [11]:
len(interactions['rating'])

1132367

In [12]:
tag_set = set()
for row in recipes['tags']:
    for tag in row:
        tag_set.add(tag)

In [13]:
# tag_set

In [14]:
dataset = Dataset()
dataset.fit(interactions['user_id'], interactions['recipe_id'], item_features=tag_set)

In [15]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 226570, num_items 231637.


In [16]:
# dataset.fit_partial(items=interactions['recipe_id'],
#                     item_features=recipes['features'])

In [ ]:
from sklearn.model_selection import train_test_split

interactions_train, interactions_test = train_test_split(interactions, test_size=0.20)

In [ ]:
(interactions_matrix, weights) = dataset.build_interactions(interactions_train[['user_id', 'recipe_id', 'rating']].values.tolist())

print(repr(interactions_matrix))

In [ ]:
# t = recipes[['id', 'tags']][:5]
# t['tags']=t['tags'].apply(lambda x: string_list_to_list(x))
# t

In [ ]:
item_features = dataset.build_item_features(recipes[['id', 'tags']].values.tolist())
print(repr(item_features))

In [ ]:
from lightfm import LightFM

model = LightFM(loss='bpr')
# model.fit(interactions_matrix, item_features=item_features, epochs=3)
model.fit(interactions_matrix, item_features=item_features)

In [ ]:
len(recipes['id'].unique())

In [ ]:
interactions_train.head()

In [ ]:
interactions['user_id'].unique()[:5]

In [ ]:
for user_id in interactions_test['user_id'].unique():
    n_recipes = len(recipes['id'].unique())
    known_positives = interactions_train.loc[(interactions_train['user_id'] == user_id) & (interactions_train['rating'] == 5)]
    known_positives = pd.merge(known_positives, recipes, left_on="recipe_id", right_on='id')

    scores = model.predict(int(user_id), np.arange(n_recipes))
    print(scores[:10])
    
    top_items = recipes[['id', 'name','tags']].sort_values(by='id')
    top_items['scores'] = scores
    top_items.sort_values(by='scores', ascending=False)

    print("User %s" % user_id)
    print("     Known positives:")
    
    print(known_positives[['name', 'tags']][:10])

    print("     Recommended:")
    
    print(top_items[['name', 'tags', 'scores']][:10])